- Distribución uniforme discreta.
- Distribución de Bernoulli.
- Distribución binomial.
- Distribución de Polsson.

- Distribución multinomial.
- Distribución geométríca.
- Distribución binomial negativa.
- Distribución hipergeométríca.

In [1]:
import numpy as np
from scipy.stats import bernoulli, binom, poisson, multinomial, geom, nbinom, hypergeom

np.random.seed(42)

# 1) Distribución uniforme discreta (dado justo)
valores = np.arange(1, 7)
pmf_uniforme = {v: 1/6 for v in valores}
print("Uniforme discreta: P(X=3) =", pmf_uniforme[3])

# 2) Distribución de Bernoulli (éxito/fracaso)
p = 0.3
print("Bernoulli: P(X=1) =", bernoulli.pmf(1, p), "P(X=0) =", bernoulli.pmf(0, p))

# 3) Distribución binomial (n ensayos, prob p)
n, p = 10, 0.5
k = 6
print("Binomial: P(X=6) =", binom.pmf(k, n, p), "E[X] =", n*p, "Var[X] =", n*p*(1-p))

# 4) Distribución de Poisson (tasa λ)
lam = 3
k = 2
print("Poisson: P(X=2) =", poisson.pmf(k, lam), "E[X] =", lam)

# 5) Distribución multinomial (n ensayos, categorías p_i)
n = 10
ps = [0.2, 0.3, 0.5]
xs = [2, 3, 5]
print("Multinomial: P(X=[2,3,5]) =", multinomial.pmf(xs, n, ps))

# 6) Distribución geométrica (primer éxito en intento k, soporte k>=1)
p = 0.25
k = 4
print("Geométrica: P(X=4) =", geom.pmf(k, p), "E[X] =", 1/p)

# 7) Distribución binomial negativa (fallos antes de r éxitos)
r, p = 3, 0.4  # r éxitos, prob de éxito p
k = 4          # fallos
print("Binomial negativa: P(fallos=4) =", nbinom.pmf(k, r, p), "E[fallos] =", r*(1-p)/p)

# 8) Distribución hipergeométrica (sin reemplazo)
N, K, n = 50, 12, 10   # población N, éxitos K, extracciones n
x = 4
print("Hipergeométrica: P(X=4) =", hypergeom.pmf(x, N, K, n), "E[X] =", n*K/N)

# --- Verificación rápida por simulación (opcional) ---
# Binomial simulación
sim_binom = np.mean(np.random.binomial(n=10, p=0.5, size=100_000) == 6)
print("Binomial sim P(X=6) ~", round(sim_binom, 4))

# Poisson simulación
sim_pois = np.mean(np.random.poisson(lam=3, size=100_000) == 2)
print("Poisson sim P(X=2) ~", round(sim_pois, 4))

# Geométrica simulación
sim_geom = np.mean(np.random.geometric(p=0.25, size=100_000) == 4)
print("Geométrica sim P(X=4) ~", round(sim_geom, 4))

# Hipergeométrica simulación
sim_hg = np.mean(hypergeom.rvs(N, K, n, size=100_000) == 4)
print("Hipergeométrica sim P(X=4) ~", round(sim_hg, 4))

Uniforme discreta: P(X=3) = 0.16666666666666666
Bernoulli: P(X=1) = 0.3 P(X=0) = 0.7000000000000002
Binomial: P(X=6) = 0.2050781249999999 E[X] = 5.0 Var[X] = 2.5
Poisson: P(X=2) = 0.22404180765538775 E[X] = 3
Multinomial: P(X=[2,3,5]) = 0.08504999999999999
Geométrica: P(X=4) = 0.10546875 E[X] = 4.0
Binomial negativa: P(fallos=4) = 0.1244160000000001 E[fallos] = 4.499999999999999
Hipergeométrica: P(X=4) = 0.13303155078013235 E[X] = 2.4
Binomial sim P(X=6) ~ 0.2064
Poisson sim P(X=2) ~ 0.2252
Geométrica sim P(X=4) ~ 0.1046
Hipergeométrica sim P(X=4) ~ 0.1335


# Ejemplos de Distribuciones Discretas y Proceso de Poisson

Este notebook muestra ejemplos mínimos de:
- Distribución uniforme discreta
- Distribución de Bernoulli
- Distribución binomial
- Distribución de Poisson
- Distribución multinomial
- Distribución geométrica
- Distribución binomial negativa
- Distribución hipergeométrica

Al final, se resuelve el problema del tiempo de espera > 10 minutos usando propiedades del Proceso de Poisson y la distribución exponencial.

In [2]:
import numpy as np
import math
from scipy.stats import bernoulli, binom, poisson, multinomial, geom, nbinom, hypergeom
np.random.seed(42)

## 1) Distribución uniforme discreta (dado justo)
PMF: P(X=x) = 1/N para cada valor del conjunto.

In [3]:
valores = np.arange(1, 7)
pmf_uniforme = {v: 1/6 for v in valores}
print("Uniforme discreta: P(X=3) =", pmf_uniforme[3])

# Verificación por simulación
n = 200_000
muestras = np.random.choice(valores, size=n)
sim_p_3 = np.mean(muestras == 3)
print("Simulación ~ P(X=3) =", round(sim_p_3, 4))

Uniforme discreta: P(X=3) = 0.16666666666666666
Simulación ~ P(X=3) = 0.1651


## 2) Distribución de Bernoulli (éxito/fracaso)
Parámetro p: probabilidad de éxito.

In [4]:
p = 0.3
print("Bernoulli: P(X=1) =", bernoulli.pmf(1, p), "P(X=0) =", bernoulli.pmf(0, p))
print("E[X]=", p, "Var[X]=", p*(1-p))

Bernoulli: P(X=1) = 0.3 P(X=0) = 0.7000000000000002
E[X]= 0.3 Var[X]= 0.21


## 3) Distribución binomial (n ensayos con prob p)
PMF: P(X=k) = C(n,k) p^k (1-p)^(n-k)

In [5]:
n, p = 10, 0.5
k = 6
print("Binomial: P(X=6) =", binom.pmf(k, n, p), "E[X]=", n*p, "Var[X]=", n*p*(1-p))

# Simulación
sim_binom = np.mean(np.random.binomial(n=n, p=p, size=100_000) == k)
print("Simulación ~ P(X=6) =", round(sim_binom, 4))

Binomial: P(X=6) = 0.2050781249999999 E[X]= 5.0 Var[X]= 2.5
Simulación ~ P(X=6) = 0.2062


## 4) Distribución de Poisson (cuentas con tasa λ)
PMF: P(X=k) = e^{-λ} λ^k / k!

In [6]:
lam = 3
k = 2
print("Poisson: P(X=2) =", poisson.pmf(k, lam), "E[X]=", lam)

# Simulación
sim_pois = np.mean(np.random.poisson(lam=lam, size=100_000) == k)
print("Simulación ~ P(X=2) =", round(sim_pois, 4))

Poisson: P(X=2) = 0.22404180765538775 E[X]= 3
Simulación ~ P(X=2) = 0.2244


## 5) Distribución multinomial (n ensayos, categorías p_i)
Extiende la binomial a múltiples categorías.

In [7]:
n = 10
ps = [0.2, 0.3, 0.5]
xs = [2, 3, 5]
print("Multinomial: P(X=[2,3,5]) =", multinomial.pmf(xs, n, ps))

# Simulación: frecuencias de categorías
m = 50_000
sim = np.random.multinomial(n, ps, size=m)
pmf_sim = np.mean(np.all(sim == xs, axis=1))
print("Simulación ~ P(X=[2,3,5]) =", round(pmf_sim, 6))

Multinomial: P(X=[2,3,5]) = 0.08504999999999999
Simulación ~ P(X=[2,3,5]) = 0.0833


## 6) Distribución geométrica (primer éxito en intento k, k≥1)
PMF: P(X=k) = (1-p)^{k-1} p

In [8]:
p = 0.25
k = 4
print("Geométrica: P(X=4) =", geom.pmf(k, p), "E[X]=", 1/p)

# Simulación
sim_geom = np.mean(np.random.geometric(p=p, size=100_000) == k)
print("Simulación ~ P(X=4) =", round(sim_geom, 4))

Geométrica: P(X=4) = 0.10546875 E[X]= 4.0
Simulación ~ P(X=4) = 0.1045


## 7) Distribución binomial negativa (fallos antes de r éxitos)
PMF: P(K=k) = C(k+r-1, k) (1-p)^k p^r

In [9]:
r, p = 3, 0.4
k = 4  # fallos
print("Binomial negativa: P(fallos=4) =", nbinom.pmf(k, r, p), "E[fallos]=", r*(1-p)/p)

# Simulación
m = 100_000
sim_nb = nbinom.rvs(r, p, size=m)
sim_p = np.mean(sim_nb == k)
print("Simulación ~ P(fallos=4) =", round(sim_p, 4))

Binomial negativa: P(fallos=4) = 0.1244160000000001 E[fallos]= 4.499999999999999
Simulación ~ P(fallos=4) = 0.124


## 8) Distribución hipergeométrica (sin reemplazo)
Población N, éxitos K, extracciones n.

In [10]:
N, K, n = 50, 12, 10
x = 4
print("Hipergeométrica: P(X=4) =", hypergeom.pmf(x, N, K, n), "E[X]=", n*K/N)

# Simulación
sim_hg = np.mean(hypergeom.rvs(N, K, n, size=100_000) == x)
print("Simulación ~ P(X=4) =", round(sim_hg, 4))

Hipergeométrica: P(X=4) = 0.13303155078013235 E[X]= 2.4
Simulación ~ P(X=4) = 0.1327


---
## Problema: Espera > 10 minutos entre llamadas (Exponencial / Poisson)
Tasa λ = 4 llamadas/hora. Pregunta: P(T > 10 min)?
- En exponencial: P(T>t) = e^{-λ t}
- En Poisson: "esperar > 10 min" equivale a 0 llamadas en 10 min → N~Poisson(μ), μ=λ·t; P(N=0)=e^{-μ}

In [11]:
import math
lam = 4            # llamadas por hora
t_hr = 10/60       # 10 minutos en horas
mu = lam * t_hr    # media de Poisson en 10 min

p_exponencial = math.exp(-lam * t_hr)
p_poisson_0 = math.exp(-mu)  # igual a p_exponencial
print(f"μ = {mu:.6f}")
print(f"P(T>10 min) (Exponencial) = {p_exponencial:.6f} ({p_exponencial*100:.2f}%)")
print(f"P(0 llamadas en 10 min) (Poisson) = {p_poisson_0:.6f} ({p_poisson_0*100:.2f}%)")

# Verificación por simulación
M = 1_000_000
muestras = np.random.exponential(scale=1/lam, size=M)
p_sim = np.mean(muestras > t_hr)
print(f"Simulación ~ P(T>10 min) = {p_sim:.6f} ({p_sim*100:.2f}%)")

μ = 0.666667
P(T>10 min) (Exponencial) = 0.513417 (51.34%)
P(0 llamadas en 10 min) (Poisson) = 0.513417 (51.34%)
Simulación ~ P(T>10 min) = 0.513487 (51.35%)
